In [1]:
import pandas as pd
import numpy as np
import math

SOLVER_MILO = "highs"
SOLVER_MINLO = "ipopt"

from amplpy import AMPL, ampl_notebook

ampl = ampl_notebook(
    modules=["coin", "highs"],  # modules to install
    license_uuid="default",  # license to use
)  # instantiate AMPL object and register magics

AMPL Development Version 20240404 (MSVC 19.38.33135.0, 64-bit)
Demo license with maintenance expiring 20260131.
Using license file "c:\Users\thuduong\Anaconda3\envs\optima\Lib\site-packages\ampl_module_base\bin\ampl.lic".



In [33]:
## PARAMETER
# over cut bound
BOUND = 0.3
MIN_MARGIN = 8

# DATA TEST 1
stocks = {
    "S1": {"width": 1219, "weight": 4395 },
    "S2": {"width": 1219, "weight": 9260},
    "S3": {"width": 1018, "weight": 3475},
    # "S4": {"width": 1219, "weight": 8535},
    "S5": {"width": 236, "weight": 1571},
}

finish = {
    "F1": {"width": 235, "need_cut": 800 },
    "F2": {"width": 147, "need_cut": 1308},
    "F3": {"width": 136, "need_cut": 1290},
    "F4": {"width": 68, "need_cut": 600},
    "F5": {"width": 60, "need_cut": 170},
    "F6": {"width": 85, "need_cut": 132},
    "F7": {"width": 57, "need_cut": 100},
    "F8": {"width": 92, "need_cut": 100}, 
    "F9": {"width": 57, "need_cut": 735}, 
}

# uu tien cat het 1 finished goods trong 1 cuon thep
# -> need cut chuyen thanh duong truoc khi sang finished goods khac

## 0.Problem with known naive patterns

Given a list of patterns, the optimization problem is to compute how many copies of each pattern should be cut to fit the width of the Mother Coil and under the weight demand of the Finished Goods

A pattern $p$ is specified by the stock $s_p$ assigned to the pattern and <br>
integers $ap_{f}$ that specify the maximum of the finished parts of type $f$ are cut from stock $s_p$ and <br>
$\text{margin}^S_{s}$ is the allowed minimum margin of the according steel coil type; <br>
$wu^S_{s}$ the weight unit per mm wight of the coil stock, which is the weight divided by the MC width; <br>
$d^F_f$ the weight demand of the Finished Goods. A pattern $p\in P$ is feasible if

$$
\begin{align}
& ap_{f}l^F_f  \leq   l^S_{s} - \text{margin}^S_{s} && \forall f\in F\\
& wu_{s} \times ( ap_{f} l^F_f) \leq d^F_f && \forall f\in F
\end{align}
$$

In [3]:
from codes.create_patterns import *

naive_patterns = make_patterns_by_weight_width(stocks, finish, BOUND, MIN_MARGIN)

# display(naive_patterns)
ap_upper_bound(naive_patterns,finish)

No feasible pattern was found for Stock S4 and FG F6
No feasible pattern was found for Stock S4 and FG F7
No feasible pattern was found for Stock S4 and FG F8


{'F1': 1,
 'F2': 4,
 'F3': 4,
 'F4': 4,
 'F5': 1,
 'F6': 0,
 'F7': 0,
 'F8': 0,
 'F9': 5}

In [38]:
patterns = []

for s in stocks:
    max_cuts_dict, min_cuts_dict = ap_stock_bound(naive_patterns,finish,s)
    generate_cut_combinations(s, min_cuts_dict, max_cuts_dict, patterns)

[{'stock': 'S3', 'cuts': {'F4': 0, 'F2': 0, 'F5': 0, 'F3': 0, 'F1': 1, 'F6': 0, 'F9': 0}}, {'stock': 'S3', 'cuts': {'F4': 0, 'F2': 0, 'F5': 0, 'F3': 1, 'F1': 1, 'F6': 0, 'F9': 0}}, {'stock': 'S3', 'cuts': {'F4': 0, 'F2': 0, 'F5': 0, 'F3': 2, 'F1': 1, 'F6': 0, 'F9': 0}}, {'stock': 'S3', 'cuts': {'F4': 0, 'F2': 0, 'F5': 0, 'F3': 3, 'F1': 1, 'F6': 0, 'F9': 0}}, {'stock': 'S3', 'cuts': {'F4': 0, 'F2': 0, 'F5': 0, 'F3': 4, 'F1': 1, 'F6': 0, 'F9': 0}}, {'stock': 'S3', 'cuts': {'F4': 0, 'F2': 1, 'F5': 0, 'F3': 0, 'F1': 1, 'F6': 0, 'F9': 0}}, {'stock': 'S3', 'cuts': {'F4': 0, 'F2': 1, 'F5': 0, 'F3': 1, 'F1': 1, 'F6': 0, 'F9': 0}}, {'stock': 'S3', 'cuts': {'F4': 0, 'F2': 1, 'F5': 0, 'F3': 2, 'F1': 1, 'F6': 0, 'F9': 0}}, {'stock': 'S3', 'cuts': {'F4': 0, 'F2': 1, 'F5': 0, 'F3': 3, 'F1': 1, 'F6': 0, 'F9': 0}}, {'stock': 'S3', 'cuts': {'F4': 0, 'F2': 1, 'F5': 0, 'F3': 4, 'F1': 1, 'F6': 0, 'F9': 0}}]


Here, we will assume there exists the optimal solution in the combination created from the naive pattern

The function `make_patterns` defined below produces a partial list of feasible patterns for given sets of stocks and finished parts. Each pattern is represented as dictionary that specifies an associated stock item, and a dictionary of cuts that specify the finished parts cut from the stock. The algorithm is simple, it just considers every finished parts and stock items, then reports the number of parts $f$ that can be cut from stock item $s$.

The `cut_patterns` model requires a known list of cutting patterns. This works well if the patterns comprising an optimal solution to the problem are known. But since they are not initially known, an optimization model is needed that simultaneously solves for an optimal patterns and the cutting list.

## 1.Cutting Stock Problem:
Let binary variable $b_{sp}\in\mathbb{Z}_2$ denote the assignment of stock $s$ to pattern $p$, and let $P = 0, 1, \ldots, N_p-1$ index a list of patterns, $w^S_{s}$ the weight unit per mm wight of the coil stock, which is the weight divided by the width. This solution is attempted using a mixed-integer nonlinear optimization (MINLO) solver.

For sufficiently large $N_p$, an optimal solution to the stock cutting problem is given by the model

In [29]:
def cut_pattern_bilinear(stocks, finish, Np, pattern_prob):
    # Np=len(finish) # CAI NAY CO ANH HUONG GI

    pattern_prob.eval("reset data;")
    pattern_prob.set["S"] = list(stocks.keys())
    pattern_prob.set["F"] = list(finish.keys())
    pattern_prob.set["P"] = list(range(Np)) # list number of patterns
 
    pattern_prob.param["wu"] = {s: stocks[s]["weight"]/stocks[s]["width"] for s in list(stocks.keys())} # stock weight per unit (unique)
    pattern_prob.param["width_s"] = {s: stocks[s]["width"] for s in stocks.keys()}
    
    pattern_prob.param["width_f"] = {f: finish[f]["width"] for f in finish.keys()}
    pattern_prob.param["demand_finish"] = {f: finish[f]["need_cut"] for f in finish.keys()}
    pattern_prob.param["f_upper_demand"] = {f: finish[f]["need_cut"] + BOUND * finish[f]["need_cut"] for f in finish.keys()}

    pattern_prob.param["a_upper_bound"] = ap_upper_bound(naive_patterns,finish)
    pattern_prob.get_output("solve;")

    return pattern_prob


### TEST 1

\begin{align}
\min \text{trim cost}: \quad & \sum_{s\in S} b_{sp} l^S_s - \sum_{f\in F}a_{fp}l^F_f && \forall p\in P\\
\text{s.t.} \quad
& \sum_{s\in S}b_{sp} =  1 && \forall p\in P \\
& 96\% \times \sum_{s\in S}b_{sp} l^S_s \leq \sum_{f\in F}a_{fp}l^F_f \leq \sum_{s\in S}b_{sp} l^S_s -\text{margin}_{s} && \forall p\in P \\
& \sum_{s \in S}b_{sp} wu^S_{s} \times (a_{fp} l^F_{f})  \leq d^F_f && \forall f\in F, \forall p\in P \\

& a_{fp} \in \mathbb{Z}_+ && \forall f\in F,  \forall p\in P , \\
& b_{sp} \in \{0,1\} && \forall s\in S,  \forall p\in P \\
\end{align}

### 1.1.Explaination:
##### A. Pattern Constraint
For each *Pattern*, find one stock that fit the requirement of minimization
$$
\sum_{s\in S}b_{sp} =  1 \quad \forall p\in P 
$$
##### B. Width Demaind Constraint
The width of the finished goods cut for each item $f \in F$ 
should be at least 96% of the width of the mother coil (with a maximum 4% trim loss), but no greater than the width of the mother coil minus the stock margin."
$$
96\% \times \sum_{s\in S}b_{sp} l^S_s \leq \sum_{f\in F}a_{fp}l^F_f \leq \sum_{s\in S}b_{sp} l^S_s -\text{margin}_{s} \quad \forall p\in P 
$$
##### C. Weight Demand Constraint
Find, for each *Pattern*, the stock whose weight ensures that the combined weight of the after-cut pieces is either less than or equal to the upper-bound weight of the Finished Goods.
$$
\sum_{s \in S} b_{sp} wu^S_{s} \times (a_{fp} l^F_{f}) \leq d_f \quad \forall p \in P ,  \forall f \in F 
$$

In [30]:
##### TEST 1
# problem to find the stock with pattern that fits the weight
#because heavier stock (same width) can also fit the pattern
# Find the patterns of stock that minimize the loss BY PATTERN
m = AMPL()
m.option["solver"] = SOLVER_MINLO
m.eval(
    """
        set S;
        set F;
        set P;

        # width stock
        param width_s{S};
        # width finished pieces
        param width_f{F};
        
        # weight per unit of stock
        param wu{S};

        # upper bound with over-cut
        param f_upper_demand{F};
        param demand_finish{F};
        param a_upper_bound{F};

        # how many f pieces are returned from pattern p
        var a{f in F, p in P} integer >= 0 <= a_upper_bound[f];
        # which stock s is choosen for pattern p
        var b{S, P} binary;

        # Find the patterns of stock that minimize the loss
        minimize trim_loss {p in P}:
          sum{s in S} b[s, p] * width_s[s] - sum{f in F} a[f,p] * width_f[f];
        
        subject to assign_each_stock_to_pattern {p in P}:
          sum{s in S} b[s, p]  = 1;
        
        subject to feasible_pattern_max_margin {p in P}:
          sum{f in F} a[f,p] * width_f[f] >= 0.96 * sum{s in S} b[s,p] * width_s[s];
        
        subject to feasible_pattern_min_margin {p in P}:
          sum{f in F} a[f,p] * width_f[f] <= sum{s in S} b[s,p] * width_s[s] - 8;

        # subject to weight_demand {p in P, f in F}:
        #   a[f,p] * width_f[f] * sum{s in S} b[s, p] * wu[s] <= f_upper_demand[f];
    """
)

In [34]:
# Np=len(naive_patterns)
Np = 100
solved_prob = cut_pattern_bilinear(stocks, finish, Np, m)

# Retrieve the patterns
patterns = []
for i, s in enumerate(stocks):
    try:
        patterns.append({
            "stock": [(s, p) for p in range(Np) if solved_prob.var["b"][s, p].value() > 0][0],
            # "patterns": {f: round(solved_prob.var["a"][f, p].value()) for p in range(Np) if solved_prob.var["b"][s, p].value() > 0 and f in finish.keys()}
        })
    except:
        print(f"Error: Cannot find optimized pattern for stock {s}")


Error: Cannot find optimized pattern for stock S1
Error: Cannot find optimized pattern for stock S2
Error: Cannot find optimized pattern for stock S3
Error: Cannot find optimized pattern for stock S5


In [32]:
# Given dictionaries of stocks and finished parts, and a list of patterns,
# find minimum choice of patterns to cut

def cut_patterns_width(stocks, finish, patterns):
    m = AMPL()

    m.eval(
    """
        set S;
        set F;
        set P;

        param a{F, P};
        # width stock
        param width_s{S};
        # width finished pieces
        param width_f{F};

        # which stock s is choosen for pattern p
        var b{S, P} binary;

        minimize trim_loss {p in P}:
            sum{s in S} b[s, p] * width_s[s] - sum{f in F} a[f,p] * width_f[f];
        
        subject to assign_each_stock_to_pattern {p in P}:
            sum{s in S} b[s, p]  = 1;

        subject to feasible_pattern_min_margin {p in P}:
            sum{f in F} a[f,p] * width_f[f] <= sum{s in S} b[s,p] * width_s[s] - 8;
    """
    )

    m.set["S"] = list(stocks.keys())
    m.set["F"] = list(finish.keys())
    m.set["P"] = list(range(len(patterns)))

    # s = {p: patterns[p]["stock"] for p in range(len(patterns))}
    a = {
        (f, p): patterns[p]["cuts"][f]
        for p in range(len(patterns))
        for f in finish.keys()
    }
    m.param["a"] = a

    m.option["solver"] = SOLVER_MILO
    m.get_output("solve;")

    return [m.var["b"][p].value() for p in range(len(patterns))]

x, cost = cut_patterns_width(stocks, finish, naive_patterns)
x

[]

### TEST 2
Remove upper bound for B. Width Demaind Constraint, as it could be satisfied in the naive pattern already

\begin{align}
\min \text{trim cost}: \quad & \sum_{s\in S} b_{sp} l^S_s - \sum_{f\in F}a_{fp}l^F_f && \forall p\in P\\
\text{s.t.} \quad
& \sum_{s\in S}b_{sp} =  1 && \forall p\in P \\
& 96\% \times \sum_{s\in S}b_{sp} l^S_s \leq \sum_{f\in F}a_{fp}l^F_f && \forall p\in P \\
& \sum_{s \in S}b_{sp} wu^S_{s} \times (a_{fp} l^F_{f})  \leq d^F_f && \forall p\in P,  \forall f\in F \\

& a_{fp} \in \mathbb{Z}_+ && \forall f\in F,  \forall p\in P , \\
& b_{sp} \in \{0,1\} && \forall s\in S,  \forall p\in P \\
\end{align}



In [ ]:
##### TEST 1

# problem to find the stock with pattern that fits the weight
#because heavier stock (same width) can also fit the pattern
# Find the patterns of stock that minimize the loss BY PATTERN

m1 = AMPL()
m1.option["solver"] = SOLVER_MINLO
m1.eval(
    """
       set S;
        set F;
        set P;

        # length stock
        param width_s{S};
        # length finished pieces
        param width_f{F};
        
        # WEIGHT STOCK
        param w{S};
        # weight per unit of stock
        param wu{S};

        # upper bound with over-cut
        param f_upper_demand{F};
        param demand_finish{F};
        param a_upper_bound{F};

        # how many f pieces are returned from pattern p
        var a{f in F, p in P} integer >= 0 <= a_upper_bound[f];
        # which stock s is choosen for pattern p
        var b{S, P} binary;

        # Find the patterns of stock that minimize the loss
        minimize trim_loss {p in P}:
          sum{s in S} b[s, p] * width_s[s] - sum{f in F} a[f,p] * width_f[f];
        
        subject to assign_each_stock_to_pattern {p in P}:
          sum{s in S} b[s, p]  = 1;
        
        subject to feasible_pattern_max_margin {p in P}:
          sum{f in F} a[f,p] * width_f[f] >= 0.96 * sum{s in S} b[s,p] * width_s[s];
        
        # subject to feasible_pattern_min_margin {p in P}:
        #   sum{f in F} a[f,p] * width_f[f] <= sum{s in S} b[s,p] * width_s[s] - 8;

        subject to demand {p in P, f in F}:
          a[f,p] * width_f[f] * sum{s in S} b[s, p] * wu[s] <= f_upper_demand[f];
    """
)

In [ ]:
Np=len(naive_patterns)
solved_prob = cut_pattern_bilinear(stocks, finish, Np, m)

# Retrieve the patterns
patterns = []
for i, s in enumerate(stocks):
    try:
        patterns.append({
            "stock": [(s, p) for p in range(Np) if solved_prob.var["b"][s, p].value() > 0][0],
            # "patterns": {f: round(solved_prob.var["a"][f, p].value()) for p in range(Np) if solved_prob.var["b"][s, p].value() > 0 and f in finish.keys()}
        })
    except:
        print(f"Error: Cannot find optimized pattern for stock {s}")
